In [0]:
%sql
SELECT * FROM datamodeling.bronze.bronze_table


order_id,order_date,customer_id,customer_name,custumee_email,product_id,product_name,product_category,quantity,unit_price,payment_type,country,last_updated
1001,2025-07-01,1,Alice Johnson,alice@example.com,501,Samsung A45,Electronics,1,1599.99,Credit Card,RSA,2025-07-01
1002,2025-07-02,2,Dimakatso Malpe,dimakatso@example.com,502,iPhone 14,Electronics,2,2689.99,PayPal,RSA,2025-07-02
1003,2025-07-03,3,Lego Mojapelo,lego@example.com,503,PlayStation 5,Electronics,1,499.99,Apple Pay,RSA,2025-07-03


In [0]:
spark.sql("""
SELECT *,
split(customer_name, ' ')[0] AS first_name,
split(customer_name, ' ')[1] AS last_name,
date(current_timestamp()) AS processDate
FROM datamodeling.bronze.bronze_table""").createOrReplaceTempView("silver_source")

In [0]:
%sql
SELECT * FROM silver_source

order_id,order_date,customer_id,customer_name,custumee_email,product_id,product_name,product_category,quantity,unit_price,payment_type,country,last_updated,first_name,last_name,processDate
1004,2025-07-04,4,Thando Ndlovu,thando@example.com,504,Dell XPS 13,Electronics,1,1299.99,Credit Card,RSA,2025-07-04,Thando,Ndlovu,2025-07-14
1005,2025-07-05,5,Sipho Mokoena,sipho@example.com,505,Nike Air Max,Apparel,2,299.99,PayPal,RSA,2025-07-05,Sipho,Mokoena,2025-07-14
1006,2025-07-06,6,Karabo Sithole,karabo@example.com,506,"Samsung TV 55""",Electronics,1,899.99,Credit Card,RSA,2025-07-06,Karabo,Sithole,2025-07-14
1007,2025-07-07,7,Boitumelo Dlamini,boitumelo@example.com,507,Adidas Hoodie,Apparel,3,149.99,Google Pay,RSA,2025-07-07,Boitumelo,Dlamini,2025-07-14
1008,2025-07-08,8,Lehlohonolo Mabuza,lehlohonolo@example.com,508,Canon EOS 90D,Electronics,1,1099.99,Credit Card,RSA,2025-07-08,Lehlohonolo,Mabuza,2025-07-14
1009,2025-07-09,9,Naledi Khumalo,naledi@example.com,509,Zara Jacket,Apparel,1,199.99,PayPal,RSA,2025-07-09,Naledi,Khumalo,2025-07-14
1010,2025-07-10,10,Tshepo Mthembu,tshepo@example.com,510,"MacBook Pro 14""",Electronics,1,2499.99,Apple Pay,RSA,2025-07-10,Tshepo,Mthembu,2025-07-14
1011,2025-07-11,11,Refilwe Nkosi,refilwe@example.com,511,iPad Air,Electronics,2,799.99,Google Pay,RSA,2025-07-11,Refilwe,Nkosi,2025-07-14
1012,2025-07-12,12,Mpho Sekhukhune,mpho@example.com,512,Reebok Classic,Apparel,1,119.99,Credit Card,RSA,2025-07-12,Mpho,Sekhukhune,2025-07-14
1013,2025-07-13,13,Kagiso Motsepe,kagiso@example.com,513,GoPro Hero 12,Electronics,1,499.99,PayPal,RSA,2025-07-13,Kagiso,Motsepe,2025-07-14


### Merge using pyspark

In [0]:
if spark.catalog.tableExists('datamodeling.silver.silver_table'):
    src = spark.sql("""select * from silver_source""")
else:
    spark.sql("""
    CREATE TABLE IF NOT EXISTS datamodeling.silver.silver_table
    AS
    SELECT * FROM silver_source
    """)

### Merge using sql

In [0]:
%sql
CREATE TABLE IF NOT EXISTS datamodeling.silver.silver_table
    AS
    SELECT * FROM silver_source

num_affected_rows,num_inserted_rows


In [0]:
%sql
MERGE INTO datamodeling.silver.silver_table
USING silver_source
ON datamodeling.silver.silver_table.order_id = silver_source.order_id
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *

num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
17,0,0,17


In [0]:
%sql
SELECT * FROM datamodeling.silver.silver_table

order_id,order_date,customer_id,customer_name,custumee_email,product_id,product_name,product_category,quantity,unit_price,payment_type,country,last_updated,first_name,last_name,processDate
1004,2025-07-04,4,Thando Ndlovu,thando@example.com,504,Dell XPS 13,Electronics,1,1299.99,Credit Card,RSA,2025-07-04,Thando,Ndlovu,2025-07-14
1005,2025-07-05,5,Sipho Mokoena,sipho@example.com,505,Nike Air Max,Apparel,2,299.99,PayPal,RSA,2025-07-05,Sipho,Mokoena,2025-07-14
1006,2025-07-06,6,Karabo Sithole,karabo@example.com,506,"Samsung TV 55""",Electronics,1,899.99,Credit Card,RSA,2025-07-06,Karabo,Sithole,2025-07-14
1007,2025-07-07,7,Boitumelo Dlamini,boitumelo@example.com,507,Adidas Hoodie,Apparel,3,149.99,Google Pay,RSA,2025-07-07,Boitumelo,Dlamini,2025-07-14
1008,2025-07-08,8,Lehlohonolo Mabuza,lehlohonolo@example.com,508,Canon EOS 90D,Electronics,1,1099.99,Credit Card,RSA,2025-07-08,Lehlohonolo,Mabuza,2025-07-14
1009,2025-07-09,9,Naledi Khumalo,naledi@example.com,509,Zara Jacket,Apparel,1,199.99,PayPal,RSA,2025-07-09,Naledi,Khumalo,2025-07-14
1010,2025-07-10,10,Tshepo Mthembu,tshepo@example.com,510,"MacBook Pro 14""",Electronics,1,2499.99,Apple Pay,RSA,2025-07-10,Tshepo,Mthembu,2025-07-14
1011,2025-07-11,11,Refilwe Nkosi,refilwe@example.com,511,iPad Air,Electronics,2,799.99,Google Pay,RSA,2025-07-11,Refilwe,Nkosi,2025-07-14
1012,2025-07-12,12,Mpho Sekhukhune,mpho@example.com,512,Reebok Classic,Apparel,1,119.99,Credit Card,RSA,2025-07-12,Mpho,Sekhukhune,2025-07-14
1013,2025-07-13,13,Kagiso Motsepe,kagiso@example.com,513,GoPro Hero 12,Electronics,1,499.99,PayPal,RSA,2025-07-13,Kagiso,Motsepe,2025-07-14
